In [ ]:
import glob
import wradlib as wrl
import matplotlib.pyplot as plt
import matplotlib as mpl
import warnings
warnings.filterwarnings('ignore')

from osgeo import osr

import pickle

import pyart
import datetime as dt
import numpy as np
import pandas as pd
import os

import sys

from matplotlib.collections import PatchCollection
from matplotlib.colors import from_levels_and_colors
import matplotlib.patches as patches



In [ ]:
# Read the radar grid coordinates. These coordinates are in WGS84 CRS.
# Getting back the objects:
with open('/home/icrisologo/SAVEUR/radar_grid.pkl','rb') as f:  # Python 3: open(..., 'rb')
    x_rad, y_rad = pickle.load(f)

def testplot(cats, catsavg, xy, data, savefname,
             levels=[0, 1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50, 100, 150],
             title=""):
    """Quick test plot layout for this example file
    """
    colors = plt.cm.viridis(np.linspace(0, 1, len(levels)))
    mycmap, mynorm = from_levels_and_colors(levels, colors, extend="max")

    radolevels = [0, 1, 2, 3, 4, 5, 10, 15, 20, 25, 30, 40, 50, 100, 150]
    radocolors = pyart.graph.cm_colorblind.HomeyerRainbow(np.linspace(0, 1, len(radolevels)))
    #pyart.graph.cm_colorblind()
    radocmap, radonorm = from_levels_and_colors(radolevels, radocolors,
                                                extend="max")

    fig = plt.figure(figsize=(18, 10))

    # Average rainfall sum
    ax = fig.add_subplot(122, aspect="equal")
    coll = PatchCollection(cats, array=catsavg, cmap=radocmap, norm=mynorm,
                           edgecolors='white', lw=0.5)
    ax.add_collection(coll)
    ax.autoscale()
    plt.colorbar(coll, ax=ax, shrink=0.5)
    plt.xlabel("NAD83 Z16 Easting")
    plt.ylabel("NAD83 Z16 Northing")
    plt.title(title)
    plt.draw()


    # Original radar data
    ax1 = fig.add_subplot(121, aspect="equal")
    pm = plt.pcolormesh(xy[:, :, 0], xy[:, :, 1], np.ma.masked_invalid(data),
                        cmap=radocmap, norm=radonorm)
    coll = PatchCollection(cats, facecolor='None', edgecolor='k', lw=0.1)
    ax1.add_collection(coll)
    cb = plt.colorbar(pm, ax=ax1, shrink=0.5)
    cb.set_label("(mm/h)")
    plt.xlabel("NAD83 Z16 Easting")
    plt.ylabel("NAD83 Z16 Northing")
    plt.title("Original radar rain sums: " + savefname.strip('.png'))
    ax1.set_xlim(bbox['left']-10000,bbox['right']+10000)
    ax1.set_ylim(bbox['bottom']-10000,bbox['top']+10000)
    #plt.draw()
    plt.tight_layout()
    fig.savefig(savefname)
    plt.close()


In [ ]:

# Read the watershed shapefile.
# this file is in UTM
fname_shp = '/home/icrisologo/Data/city_detailed_utm.shp'

dataset, inLayer = wrl.io.open_vector(fname_shp)
borders, keys = wrl.georef.get_vector_coordinates(inLayer, key='node_id')


# Define different projections that will be used in the processing.

proj_wgs = osr.SpatialReference()
proj_wgs.ImportFromEPSG(4326)

proj_aeqd = osr.SpatialReference()
proj_aeqd.ImportFromEPSG(54032)

proj_IL = osr.SpatialReference()
proj_IL.ImportFromEPSG(26771)

proj_IL_UTM = osr.SpatialReference()
proj_IL_UTM.ImportFromEPSG(26916)


# Put the two coordinate grids in one array.

grid_xy = np.zeros((300,300,2))

grid_xy[:,:,0] = x_rad
grid_xy[:,:,1] = y_rad


In [ ]:
# Reproject the radar grid to UTM, to match the shapefile.

grid_xy_utm = wrl.georef.reproject(grid_xy,
                                projection_source=proj_wgs,
                                projection_target=proj_IL_UTM)

x_rad_utm = grid_xy_utm[:,:,0]
y_rad_utm = grid_xy_utm[:,:,1]


# Create a mask to reduce size.

# Reduce grid size using a bounding box (to enhancing performance)
bbox = inLayer.GetExtent()

buffer = 5000.
bbox = dict(left=bbox[0] - buffer, right=bbox[1] + buffer,
            bottom=bbox[2] - buffer, top=bbox[3] + buffer)
mask = (((grid_xy_utm[..., 1] > bbox['bottom']) & (grid_xy_utm[..., 1] < bbox['top'])) &
        ((grid_xy_utm[..., 0] > bbox['left']) & (grid_xy_utm[..., 0] < bbox['right'])))

# Create vertices for each grid cell
# (MUST BE DONE IN NATIVE COORDINATES)

grdverts = wrl.zonalstats.grid_centers_to_vertices(x_rad_utm[mask],
                                                   y_rad_utm[mask], 824,
                                                   824)



In [ ]:
# Create instance of type ZonalDataPoly from source grid and catchment array. This needs to be done only once, then dump to file. The files that will be saved are the `src` shapefile (radar pixels), `trg` shapefile (original catchment), and `dst` shapefile (the radar pixels cut according to `trg`).

# Create instance of type ZonalDataPoly from source grid and
# catchment array
zd = wrl.zonalstats.ZonalDataPoly(grdverts, borders)

# dump to file
zd.dump_vector('zonal_poly_cart_test')

# Create instance of type ZonalStatsPoint from zonal data object
obj3 = wrl.zonalstats.ZonalStatsPoly(zd)


# ### Now the looping can happen

# Read the gridded data.
#datefolder = '2013/06/26'
datefolder = sys.argv[1]
file_dir = "/lcrc/group/earthscience/icrisologo/SAVEUR/gridded/"+datefolder+"/"
gridded_files = glob.glob(file_dir+'*')
gridded_files.sort()


df_rZ = pd.DataFrame()
f = gridded_files[0]
grid = pyart.io.read_grid(f)

data_rZ = grid.fields['radar_rainfall_depth_rainrate']['data'].squeeze()#[mask] # mask data to reduce size
data_rZ_ = data_rZ[mask]


# Compute stats for target polygons
avg_rZ = obj3.mean(data_rZ_.ravel())

# Target polygon patches
trg_patches = [patches.Polygon(item, True) for item in obj3.zdata.trg.data]



In [ ]:
# get time
dtime = str(dt.datetime.strptime(f.rsplit('/')[-1].rsplit('.')[0],'radar_KLOT_%Y%m%d_%H%M%S_gridded'))
# put to dataframe
df_rZ_ = pd.DataFrame(data=[avg_rZ],columns=keys)

df_rZ_['DateTime'] = dtime

df_rZ = df_rZ.append(df_rZ_)

# set save filename
savefname_rZ = f.rsplit('/')[-1].rsplit('.')[0]+'_rZ.png'

# plot
testplot(trg_patches, avg_rZ, grid_xy_utm, data_rZ, savefname_rZ,
     title="Catchment rainfall mean (ZonalStatsPoly)")


In [ ]:
# create empty dataframe where we will put the extracted values
df_rZ = pd.DataFrame()
df_rA = pd.DataFrame()
df_rKDP = pd.DataFrame()
df_rZ_RT = pd.DataFrame()
df_rZ_MP = pd.DataFrame()
df_rZ_WC = pd.DataFrame()

for f in gridded_files:
    print(f)
    grid = pyart.io.read_grid(f)

    data_rZ = grid.fields['radar_rainfall_depth_rainrate']['data'].squeeze()
    data_rA = grid.fields['radar_rainfall_depth_rainrate_from_attenuation']['data'].squeeze()
    data_rKDP = grid.fields['radar_rainfall_depth_rainrate_from_kdp']['data'].squeeze()
    data_rZ_RT = grid.fields['radar_rainfall_depth_rainrate_z_RT']['data'].squeeze()
    data_rZ_MP = grid.fields['radar_rainfall_depth_rainrate_z_MP']['data'].squeeze()
    data_rZ_WC = grid.fields['radar_rainfall_depth_rainrate_z_WC']['data'].squeeze()

    # mask data to reduce size
    data_rZ_ = data_rZ[mask]
    data_rA_ = data_rA[mask]
    data_rKDP_ = data_rKDP[mask]
    data_rZ_RT_ = data_rZ_RT[mask]
    data_rZ_MP_ = data_rZ_MP[mask]
    data_rZ_WC_ = data_rZ_WC[mask]

    # Compute stats for target polygons
    avg_rZ = obj3.mean(data_rZ_.ravel())
    avg_rA = obj3.mean(data_rA_.ravel())
    avg_rKDP = obj3.mean(data_rKDP_.ravel())
    avg_rZ_RT = obj3.mean(data_rZ_RT_.ravel())
    avg_rZ_MP = obj3.mean(data_rZ_MP_.ravel())
    avg_rZ_WC = obj3.mean(data_rZ_WC_.ravel())


    # Target polygon patches
    trg_patches = [patches.Polygon(item, True) for item in obj3.zdata.trg.data]

    # get time
    dtime = str(dt.datetime.strptime(f.rsplit('/')[-1].rsplit('.')[0],'radar_KLOT_%Y%m%d_%H%M%S_gridded'))
    # put to dataframe
    df_rZ_ = pd.DataFrame(data=[avg_rZ],columns=keys)
    df_rA_ = pd.DataFrame(data=[avg_rA],columns=keys)
    df_rKDP_ = pd.DataFrame(data=[avg_rKDP],columns=keys)
    df_rZ_RT_ = pd.DataFrame(data=[avg_rZ_RT],columns=keys)
    df_rZ_MP_ = pd.DataFrame(data=[avg_rZ_MP],columns=keys)
    df_rZ_WC_ = pd.DataFrame(data=[avg_rZ_WC],columns=keys)

    df_rZ_['DateTime'] = dtime
    df_rA_['DateTime'] = dtime
    df_rKDP_['DateTime'] = dtime
    df_rZ_RT_['DateTime'] = dtime
    df_rZ_MP_['DateTime'] = dtime
    df_rZ_WC_['DateTime'] = dtime

    df_rZ = df_rZ.append(df_rZ_)
    df_rA = df_rA.append(df_rA_)
    df_rKDP = df_rKDP.append(df_rKDP_)
    df_rZ_RT = df_rZ_RT.append(df_rZ_RT_)
    df_rZ_MP = df_rZ_MP.append(df_rZ_MP_)
    df_rZ_WC = df_rZ_WC.append(df_rZ_WC_)

    # set save filename
    savefname_rZ = f.rsplit('/')[-1].rsplit('.')[0]+'_rZ.png'
    savefname_rA = f.rsplit('/')[-1].rsplit('.')[0]+'_rA.png'
    savefname_rKDP = f.rsplit('/')[-1].rsplit('.')[0]+'_rKDP.png'
    savefname_rZ_RT = f.rsplit('/')[-1].rsplit('.')[0]+'_rZ_RT.png'
    savefname_rZ_MP = f.rsplit('/')[-1].rsplit('.')[0]+'_rZ_MP.png'
    savefname_rZ_WC = f.rsplit('/')[-1].rsplit('.')[0]+'_rZ_WC.png'

    # plot
    testplot(trg_patches, avg_rZ, grid_xy_utm, data_rZ, savefname_rZ,
         title="Catchment rainfall mean (ZonalStatsPoly)")
    testplot(trg_patches, avg_rA, grid_xy_utm, data_rA, savefname_rA,
         title="Catchment rainfall mean (ZonalStatsPoly)")
    testplot(trg_patches, avg_rKDP, grid_xy_utm, data_rKDP, savefname_rKDP,
         title="Catchment rainfall mean (ZonalStatsPoly)")
    testplot(trg_patches, avg_rZ_RT, grid_xy_utm, data_rZ_RT, savefname_rZ_RT,
         title="Catchment rainfall mean (ZonalStatsPoly)")
    testplot(trg_patches, avg_rZ_MP, grid_xy_utm, data_rZ, savefname_rZ_MP,
         title="Catchment rainfall mean (ZonalStatsPoly)")
    testplot(trg_patches, avg_rZ_WC, grid_xy_utm, data_rZ, savefname_rZ_WC,
         title="Catchment rainfall mean (ZonalStatsPoly)")


datestr = dt.datetime.strftime(dt.datetime.strptime(dtime,'%Y-%m-%d %H:%M:%S').date(),'%Y%m%d')

df_rZ.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rZ.csv')
df_rA.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rA.csv')
df_rKDP.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rKDP.csv')
df_rZ_RT.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rZ_RT.csv')
df_rZ_MP.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rZ_MP.csv')
df_rZ_WC.to_csv('case_study_01_'+datestr+'_hourlyrainfall_chicago_catchments_NODEIDS_rZ_WC.csv')

